In [1]:
import numpy as np
from pyrep import PyRep
from arm import CtRobot
from my_rigid_kinematics import dh_robot_config
import sympy as sp
import os
import transforms3d as t3d
from pyrep.backend import vrep

pi = np.pi

In [2]:
pr = PyRep()
pr.launch(os.getcwd() + '/Temp.ttt', headless=True)

In [3]:
pr.start()
ct_robot = CtRobot()

## My calibration (based on vrep 'reference_frame')

In [4]:
# Grab modified DH parameters from V-REP using the dummy 'DH_frame'
# Need to set up their orientation following modified DH convention
D = []
theta = []
alpha = []
a = []
L = []
jointType = ['p', 'p', 'r', 'r', 'r', 'r', 'p']
for i in range(ct_robot._num_joints):
    j = i+1
    tmp_alpha = ct_robot.DH_frames[i+1].get_orientation(relative_to=ct_robot.DH_frames[i])
    tmp_a = ct_robot.DH_frames[i+1].get_position(relative_to=ct_robot.DH_frames[i])
    if tmp_alpha[0] < -1e-5 or tmp_alpha[0] > 1e-5:
        alpha.append(tmp_alpha[0])
    else:
        alpha.append(0)
    a.append(tmp_a[0])
    
    tmp_d = -ct_robot.DH_frames[j-1].get_position(relative_to=ct_robot.DH_frames[j])[2]
    tmp_theta = -ct_robot.DH_frames[j-1].get_orientation(relative_to=ct_robot.DH_frames[j])[2]
    if tmp_theta < -1e-5 or tmp_theta > 1e-5:
        theta.append(tmp_theta)
    else:
        theta.append(0)   
    D.append(tmp_d)
# Center of mass of link with respect to each joint    
for i in range(ct_robot._num_joints+1):
    L.append(ct_robot.COMs[i].get_position(relative_to=ct_robot.DH_frames[i]))
L = np.array(L)

Tbase = np.concatenate((np.array(ct_robot.DH_frames[0].get_matrix()).reshape(3,4), np.array([[0,0,0,1]])))

num_joints = ct_robot._num_joints

In [5]:
M = np.array([
    [5.539, 5.539, 5.539, 2.152e-2, 7.838e-3, 2.866e-2],
    [6.938e-1, 6.938e-1, 6.938e-1, 9.52e-3, 2.751e-2, 3.603e-2],
    [1.196, 1.196, 1.196, 2.411e-2, 9.92e-2, 1.164e-2],
    [4.147, 4.147, 4.147, 4.954e-2, 1.441e-1, 1.814e-2],
    [3.298e-2, 3.298e-2, 3.298e-2, 3.752e-1, 2.050e-4, 3.75e-1],
    [2.375e-2, 2.375e-2, 2.375e-2, 4.592e-2, 1.718e-4, 4.591e-1],
    [3.226e-2, 3.226e-2, 3.226e-2, 5.793e-1, 0, 5.771e-1],
    [1.491e-2, 1.491e-2, 1.491e-2, 6.070e-1, 1.831e-4, 6.058e-1]
])

In [6]:
robot = dh_robot_config(num_joints, alpha, theta, D, a, jointType, Tbase, L, M)

In [7]:
robot.initKinematicTransforms()

## V-REP

In [8]:
for i in range(7):
    print('---------------------------')
    print('Joint {}'.format(i+1))
    print('D: {}, theta: {}, a[i-1]: {}, alpha[i-1]: {}'.format(D[i],theta[i],a[i],alpha[i]))

---------------------------
Joint 1
D: -0.0, theta: 0, a[i-1]: 0.0, alpha[i-1]: 0
---------------------------
Joint 2
D: 2.2900366047906573e-09, theta: 0, a[i-1]: 0.05849161744117737, alpha[i-1]: 1.570797085762024
---------------------------
Joint 3
D: 1.7881393432617188e-07, theta: 0, a[i-1]: 0.1404283195734024, alpha[i-1]: -1.5707967281341553
---------------------------
Joint 4
D: -5.424022674560547e-06, theta: 1.5707966089248657, a[i-1]: 3.4868717193603516e-06, alpha[i-1]: 1.5708012580871582
---------------------------
Joint 5
D: 1.6391277313232422e-06, theta: 0, a[i-1]: 0.07999992370605469, alpha[i-1]: 1.5708024501800537
---------------------------
Joint 6
D: 2.364599822612945e-06, theta: -1.5707964897155762, a[i-1]: 0.08000022172927856, alpha[i-1]: 1.5707963705062866
---------------------------
Joint 7
D: 0.03896474838256836, theta: 0, a[i-1]: 2.592802047729492e-06, alpha[i-1]: -1.570796251296997


## DH-FK

In [9]:
set_jot_pos = [0.01, 0.02, 0.4, 0.5, 0, 0.3, 0.01]
#set_jot_pos = [0.0, 0.0, 0.0, 0.0, 0, 0.0, 0.0]
ct_robot.set_joint_positions(set_jot_pos)

In [10]:
joint_pos = [ct_robot.joints[i].get_joint_position() for i in range(ct_robot._num_joints)]
print('Joint position is', joint_pos)
print('End-joint position is', ct_robot._ik_tip.get_position())
print('End-joint orientation is', ct_robot._ik_tip.get_orientation())

Joint position is [0.009999999776482582, 0.019999999552965164, 0.4000000059604645, 0.5, 0.0, 0.30000001192092896, 0.009999999776482582]
End-joint position is [0.053613822907209396, -0.7576621174812317, 0.11929528415203094]
End-joint orientation is [1.7553976774215698, 0.07745611667633057, -1.17795729637146]


In [11]:
err_j2j = 0

for i in range(7):
    #vrep
    T = np.eye(4)
    T[:3, :] = np.array(ct_robot.DH_frames[i+1].get_matrix(ct_robot.DH_frames[i])).reshape(3,4)
    
    # FK
    Tp = sp.lambdify(robot.q, robot._Tj2j[i])(*tuple(joint_pos))
    print('Joint %d: diff:'%(i+1), np.linalg.norm(Tp - T))
    if np.linalg.norm(Tp - T) > 1e-4:
        err_j2j += 1
        print('Wrong match of joint %d'%(i+1))
        print('getting from V-REP')
        print(T)
        print('----------------------------------')
        print('getting from FK')
        print(Tp)
        print('----------------------------------')


Joint 1: diff: 7.181045828562747e-07
Joint 2: diff: 3.422938122134113e-07
Joint 3: diff: 3.365926665740194e-05
Joint 4: diff: 2.982042506762563e-05
Joint 5: diff: 9.881946929223715e-06
Joint 6: diff: 1.3131684800142961e-05
Joint 7: diff: 5.3931767032169406e-06


In [12]:
err_j2b = 0
for i in range(7):
    #vrep
    T = np.eye(4)
    T[:3, :] = np.array(ct_robot.DH_frames[i+1].get_matrix()).reshape(3,4)
    
    # FK
    Tp = sp.lambdify(robot.q, robot._Tjoint[i])(*tuple(joint_pos))
    print('Joint %d: diff:'%(i+1), np.linalg.norm(Tp - T))
    if np.linalg.norm(Tp - T) > 1e-4:
        err_j2b += 1
        print('Wrong match of joint %d'%(i+1))
        print('getting from V-REP')
        print(T)
        print('----------------------------------')
        print('getting from FK')
        print(Tp)
        print('----------------------------------')

Joint 1: diff: 7.032553308429991e-07
Joint 2: diff: 6.245710426237736e-07
Joint 3: diff: 3.363223936045107e-05
Joint 4: diff: 4.4887476922268265e-05
Joint 5: diff: 5.132707918763116e-05
Joint 6: diff: 4.7229376975743784e-05
Joint 7: diff: 5.1495999316080035e-05


## Save

In [13]:
param = ['D', 'a', 'alpha', 'theta', 'num_joints', 'jointType', 'Tbase', 'L', 'M']
if err_j2b == 0 and err_j2j == 0:
    print('Check FK library with V-REP correct')
    print('--------------------------------------------------')
    print('Writing DH parameters, Mass and COM information into file')
    for i in range(len(param)):
        np.save('./robot_config/config_ct_7DOF/%s'%param[i], eval(param[i]))
    print('--------------------------------------------------')
    print('Writing complete')
else:
    print('Please check DH convention with V-REP')

Check FK library with V-REP correct
--------------------------------------------------
Writing DH parameters, Mass and COM information into file
--------------------------------------------------
Writing complete


In [14]:
pr.stop()
pr.shutdown()